In [9]:
import pandas as pd
import numpy as np
import datetime

In [10]:
path = 'resources/pacemed/'

df1 = pd.read_csv(path+'age.csv', index_col=None, sep=';')
df2 = pd.read_csv(path+'admission.csv', index_col=None, sep=';')
df3 = pd.read_csv(path+'signal.csv', index_col=None, sep=';')

df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)
df3.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)


In [ ]:
df2['date_admission'] = df2['date_admission'].astype('datetime64')
df2['date_discharge']= df2['date_discharge'].astype('datetime64')
df_admission = df2.groupby('pat_id')['date_admission'].agg('count').to_frame('Count').reset_index()

df_admission = pd.merge(df2, df_admission, on='pat_id', how='left')
df_admission = pd.merge(df_admission, df1, on='pat_id', how='left')
df_admission['days_ICU'] = (df_admission['date_discharge']-df_admission['date_admission']).astype('timedelta64[D]')

primary_key = df_admission['pat_id'].astype('str') +'-' + df_admission['date_admission'].astype('str')
df_admission.insert(loc=0, column='primary_key', value=primary_key)
df_admission['bad_discharge'] = [0]*df_admission.shape[0]


In [12]:
for pat in np.unique(df_admission['pat_id']):
    source_rows = df_admission.index[df_admission['pat_id']==pat].tolist()
    if(len(source_rows)>1):
        for i in range(0, len(source_rows)):
            if (i != len(source_rows)-1):
                if((df_admission['date_admission'].iloc[source_rows[i+1]] - df_admission['date_discharge'].iloc[source_rows[i]]).days < 60):
                    df_admission['bad_discharge'].iloc[source_rows[i]] = 1
df3['day'] = df3['day'].astype('datetime64')
df3.insert(loc=0, column='primary_key', value=['']*df3.shape[0])

In [13]:
for pat in np.unique(df_admission['pat_id']):
    source_rows = df_admission.index[df_admission['pat_id']==pat].tolist()
    for i in range(len(source_rows)):
        tgt_rows = df3.index[(df3['pat_id']==pat) & \
                   (df_admission['date_admission'].iloc[source_rows[i]] <= df3['day']) & \
                             (df3['day']<= df_admission['date_discharge'].iloc[source_rows[i]])].tolist()
        df3['primary_key'].iloc[tgt_rows] = [df_admission['primary_key'].iloc[source_rows[i]]]*len(tgt_rows)

In [14]:
a = list(set(np.unique(df_admission['primary_key']).tolist()) & set(np.unique(df3['primary_key']).tolist()))
len(a)
print(len(np.unique(df3['primary_key']).tolist()))
print(len(np.unique(df_admission['primary_key']).tolist()))

print(len(np.unique(df3['pat_id']).tolist()))
print(len(np.unique(df_admission['pat_id']).tolist()))

1567
1569
1492
1495


In [15]:
df_admission.to_csv(path+'test_admission.csv', sep=',',index=None)
df3.to_csv(path+'test_signal.csv', sep=',', index=None)

In [28]:
df_signal = df3.groupby(['primary_key', 'parameter']).mean()['value']
df_signal = df_signal.unstack().reset_index()
df_signal.columns.name = None
df_semifinal = df_signal.merge(df_admission[['primary_key', 'age', 'days_ICU', 'bad_discharge']], on='primary_key', how='left')
df_semifinal.to_csv(path+"collated.csv", sep=',', index=None)
df_semifinal.head()

,primary_key,blood_pressure,respiration_rate,temperature,age,days_ICU,bad_discharge
0,,NaN,NaN,36.923079,NaN,NaN,NaN
1,0-2016-01-05,133.367285,3.331333,35.746152,35.0,2.0,0.0
2,1-2014-07-04,127.878761,5.333333,37.133845,77.0,2.0,0.0
3,10-2015-07-02,109.261078,5.775556,37.405126,50.0,3.0,0.0
4,100-2014-06-28,128.925084,6.390000,36.293589,41.0,1.0,0.0


In [29]:
df_final = df_semifinal.dropna(subset=['blood_pressure','respiration_rate', 'age', 'days_ICU', 'bad_discharge'])
df_final = df_final.fillna(value=df_final['temperature'].mean())
df_final.shape



(1399, 7)

In [30]:
df_final.to_csv(path+'cleaned.csv',sep=',', index=None)